In [1]:
# -*- coding: utf-8 -*-
import sys
import codecs
import json

In [2]:
import requests
from bs4 import BeautifulSoup
from http.server import BaseHTTPRequestHandler, HTTPServer
import urllib.parse as urlparse

In [3]:
link = "https://alonhadat.com.vn/"
t = "/can-ban-nha/trang-"
r = requests.get(link+t[0])
soup = BeautifulSoup(r.text, 'html.parser')
# print(r)

In [4]:
def find_link(soup):
    l = []
    div_items = soup.find_all("div", attrs={"class" : "item"})
    for div_item in div_items:
        a_links = div_item.find_all('a',href=True)
        for a_link in a_links:
            l.append(a_link['href'])
    return l

# for value in find_link(soup):
#     print(value)

In [5]:
def find_data(soup):
    data = {}

    # Tiêu đề
    titles = soup.find_all('div', {"class": "ct_title"})
    data['title'] = titles[0].text.strip() if titles else None

    # Giá
    prices = soup.find_all('div', {"class": "ct_price"})
    data['price'] = prices[0].text.strip().replace("Gi\u00e1:","").strip() if prices else None

    # Diện tích
    acreages = soup.find_all('div', {"class": "ct_dt"})
    data['acreage'] = acreages[0].text.strip().replace("Di\u1ec7n t\u00edch:","").strip() if acreages else None

    # Mô tả
    descriptions = soup.find_all('div', {"class": "ct_brief"})
    data['description'] = descriptions[0].text.strip().replace("\r\n","") if descriptions else None

    # Ảnh
    images = soup.find_all('div', {"class": "thumbnail"})
    image_tag = images[0].find('img') if images else None
    data['image_url'] = image_tag['src'] if image_tag and 'src' in image_tag.attrs else None
    #link
    b_links = soup.find_all('a', href=True)
    data['link'] = b_links[0]['href'] if b_links else None
           
    
    return data



def process_property_elements(soup):
    data_list = []

    # Tìm tất cả các phần tử HTML chứa thông tin
    property_elements = soup.find_all('div', {"class": "content-item"})

    for property_element in property_elements:
        # Gọi hàm find_data cho mỗi phần tử và thêm kết quả vào danh sách
        data_list.append(find_data(property_element))

    return data_list

def scrape_and_save(link, t, max_pages):
    all_data = []

    for i in range(1, max_pages + 1):
        try:
            r = requests.get(link + t + str(i)+".htm")
            r.raise_for_status()  # Raise HTTPError for bad requests
            soup = BeautifulSoup(r.text, 'html.parser')
            
            property_data = process_property_elements(soup)
            all_data.extend(property_data)

        except requests.RequestException as e:
            print(f"Error fetching page {i}: {e}")

    with open('alonhadat.json', 'w') as file:
        json.dump(all_data, file)

    print("Done")

In [6]:
max_pages = 50

scrape_and_save(link, t, max_pages)

Done


In [7]:
%%javascript

const fs = require('fs');

//Đường dẫn đến tệp 'bat_dong_san.json'
const jsonFilePath = './alonhadat.json';

// Đọc nội dung của tệp JSON
fs.readFile(jsonFilePath, 'utf8', (err, data) => {
  if (err) {
    console.error(`Đã xảy ra lỗi khi đọc tệp JSON: ${err}`);
    return;
  }

  // Chuyển đổi dữ liệu JSON thành đối tượng JavaScript
  const propertyData = JSON.parse(data);

  // In ra dữ liệu để xem kết quả
  console.log(propertyData);
});

<IPython.core.display.Javascript object>